In [ ]:
import sys
import os
import time
import random
sys.path.append('../..')
from src import getSecret

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import undetected_chromedriver as uc
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
import sys
import os
import time
import random
sys.path.append('../..')
from src import getSecret

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import undetected_chromedriver as uc
from selenium.webdriver.support import expected_conditions as EC

def random_delay(start=1, end=3):
    time.sleep(random.uniform(start, end))

def send_keys_with_delay(element, text, delay_range=(0.1, 0.25)):
    """ mimic human typing ? maybe helpful maybe not necessary"""
    for char in text:
        random_delay(*delay_range)
        element.send_keys(char)

GMAIL = getSecret('GOOGLE_USERNAME')
PASSWORD = getSecret('GOOGLE_PASSWORD')

chrome_options = uc.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
# chrome_options.add_argument("--disable-extensions")
# chrome_options.add_argument("--disable-popup-blocking")
# chrome_options.add_argument("--profile-directory=Default")
# chrome_options.add_argument("--ignore-certificate-errors")
# chrome_options.add_argument("--disable-plugins-discovery")
# chrome_options.add_argument("--incognito")
# chrome_options.add_argument("user_agent=DN")
# chrome_options.add_argument("--disable-web-security")
# chrome_options.add_argument("--user-data-dir")
# chrome_options.add_argument("--allow-running-insecure-content")

driver = uc.Chrome(options=chrome_options)

driver.delete_all_cookies()

driver.get("https://myactivity.google.com/product/youtube")
random_delay()

# Find the Sign In button using the aria-label attribute
sign_in_button = driver.find_element(By.XPATH, '//*[@aria-label="Sign In"]')
sign_in_button.click()
random_delay()

email_field = driver.find_element(By.XPATH, '//*[@id="identifierId"]')
send_keys_with_delay(email_field, GMAIL)
random_delay()
email_field.send_keys(Keys.ENTER)
random_delay()

password_field = driver.find_element(By.XPATH, '//*[@id="password"]/div[1]/div/div[1]/input')
send_keys_with_delay(password_field, PASSWORD)
random_delay()
password_field.send_keys(Keys.ENTER)

try:
    # Wait for the "Try another way" button to appear and be clickable
    try_another_way_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//span[text()="Try another way"]')))
    try_another_way_button.click()
except Exception as e:
    print('No "Try another way" button')
    raise e

try:
    # Get the last two digits of the phone number
    number = getSecret('TELNYX_NUMBER')
    last_two_digits = number[-2:]
    # Wait for the list of available sign-in options to appear
    sign_in_options = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, "//div[contains(text(), 'Get a verification code at')]"))
    )
    # Find and click the option with the desired phone number
    for option in sign_in_options:
        option_text = option.text
        if last_two_digits in option_text[-2:]:
            option.click()
            break
    else:
        raise Exception('No sign-in option with the desired phone number')
except Exception as e:
    print('No "Get a verification code at" option')
    raise e


In [ ]:
try:
    try_another_way_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//span[text()="Try another way"]')))
    try_another_way_button.click()
except Exception as e:
    print('No "Try another way" button')
    raise e

In [ ]:
try:
    try_another_way_button = driver.find_element(By.XPATH, '//span[text()="Try another way"]')
    try_another_way_button.click()
except Exception as e:
    print(e)
    pass

try:
    # Get the last two digits of the phone number
    number = getSecret('TELNYX_NUMBER')
    last_two_digits = number[-2:]
    # Wait for the list of available sign-in options to appear
    sign_in_options = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, "//div[contains(text(), 'Get a verification code at')]"))
    )
    # Find and click the option with the desired phone number
    for option in sign_in_options:
        option_text = option.text
        if last_two_digits in option_text[-2:]:
            option.click()
            break
except Exception as e:
    print(e)
    pass


In [ ]:
import sys
import os
sys.path.append('../..')
from src import getSecret

import undetected_chromedriver as uc
import random,time,os,sys
from selenium.webdriver.common.keys import Keys

GMAIL = getSecret('GOOGLE_USERNAME')
PASSWORD = getSecret('GOOGLE_PASSWORD')

# Use the appropriate WebDriver for your browser
driver = webdriver.Chrome()
# driver = webdriver.Firefox()

# Navigate to the Stack Overflow login page
driver.get("https://stackoverflow.com/users/login")

# Click the "Log in with Google" button
google_login_button = driver.find_element("xpath",'//button[@data-provider="google"]')
google_login_button.click()

# Wait for the email field to appear
time.sleep(2)

# Fill in your Google email and press Enter
email_field = driver.find_element("xpath",'//*[@id="identifierId"]')
email_field.send_keys(GMAIL)
email_field.send_keys(Keys.ENTER)

# Wait for the password field to appear
time.sleep(2)

# Fill in your Google password and press Enter
password_field = driver.find_element("xpath",'//*[@id="password"]/div[1]/div/div[1]/input')
password_field.send_keys(PASSWORD)
password_field.send_keys(Keys.ENTER)

# Wait for the page to load
time.sleep(5)

# Navigate to the Google URL
driver.get("https://myactivity.google.com/product/youtube")

# The browser should now be logged in to Google and displaying the desired URL
# To close the browser, uncomment the following line
# driver.quit()


In [ ]:
import sys
import os
sys.path.append('../..')

import google.auth
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Set up API client
def get_authenticated_service():
    # Replace 'client_secrets.json' with the name of your downloaded JSON client secrets file
    flow = InstalledAppFlow.from_client_secrets_file('../../secrets/oauth2.0_youtubeAPI_secretKey.json', ['https://www.googleapis.com/auth/youtube.readonly'])
    credentials = flow.run_local_server()
    return build('youtube', 'v3', credentials=credentials)

def get_watch_history(youtube):
    try:
        results = youtube.playlistItems().list(
            part="snippet",
            playlistId="HL",  # HL is the playlist ID for watch history
            maxResults=50  # You can set this value up to 50
        ).execute()

        for item in results["items"]:
            title = item["snippet"]["title"]
            video_id = item["snippet"]["resourceId"]["videoId"]
            print(f"{title}: https://www.youtube.com/watch?v={video_id}")

    except HttpError as error:
        print(f"An error occurred: {error}")
        return None

# if __name__ == "__main__":
youtube_service = get_authenticated_service()
get_watch_history(youtube_service)
